In [1]:
import scanpy as sc
import SEACells
import numpy as np
import pandas as pd
#from sklearn.neighbors import NearestNeighbors

TypeError: metaclass conflict: the metaclass of a derived class must be a (non-strict) subclass of the metaclasses of all its bases

In [3]:
adata = sc.read('G:/Data processing pipeline 0.1 Yohan/scRNA/Output/annotation_finished_1_data_Xtrain/annotation_data_Xtrain.h5ad')

NameError: name 'sc' is not defined

In [ ]:
adata.X = adata.layers['log1p_norm_train'].copy()

In [ ]:
adata.obs['SEACell'] = 'Unk'

In [ ]:
metacell_dict = {}  # dictionary of metacells
n_SEACells  = 100    # number of meta cells per Id 
n_waypoint_eigs = 10

In [ ]:
for sample_id in adata.obs['Id'].unique():
    
   adata_sub = adata[adata.obs['Id'] == sample_id].copy() 

   model = SEACells.core.SEACells(ad, 
                  build_kernel_on='X_scVI', 
                  n_SEACells=n_SEACells, 
                  n_waypoint_eigs=n_waypoint_eigs,
                  convergence_epsilon = 1e-5)
    model.construct_kernel_matrix()

    model.initialize_archetypes()
    SEACells.plot.plot_initialization(adata_sub, model)
    model.fit(min_iter=10, max_iter=50)
    model.plot_convergence()
    adata[adata.obs['Id'] == sample_id].obs['SEACells'] = adata_sub.obs['SEACell']
    


In [ ]:
meta_test_expression = []

In [ ]:
for sample_id in adata_test.obs['Id'].unique():

    adata_train_sub = metacell_dict[sample_id]
    

    # Sélectionner les cellules X_test du même `Id`
    adata_test_sub = adata_test[adata_test.obs['Id'] == sample_id].copy()

    if adata_test_sub.n_obs == 0:
        print(f" Aucune cellule test pour {sample_id}, passage")
        continue

    # Trouver le plus proche voisin dans les métacellules correspondantes
    nn = NearestNeighbors(n_neighbors=1, metric="euclidean").fit(metacell_centers)
    distances, indices = nn.kneighbors(adata_test_sub.obsm['X_pca'])

    # Assigner chaque cellule X_test à une métacellule de X_train du même `Id`
    adata_test_sub.obs['metacell'] = [metacell_centers.index[i] for i in indices.flatten()]

    # 5. Calculer l'expression moyenne **par métacellule et par `Id`**
    meta_expr = adata_test_sub.to_df().groupby(['metacell']).mean()
    meta_expr['Id'] = sample_id  # Ajouter l'info `Id`
    meta_test_expression.append(meta_expr)


In [ ]:

for sample_id in adata_test.obs['Id'].unique():
    print(f"Projection de {sample_id}...")

    # Récupérer les métacellules de ce `Id` depuis X_train
    if sample_id not in metacell_dict:
        print(f"⚠️ Aucune métacellule définie pour {sample_id}, passage")
        continue

    adata_train_sub = metacell_dict[sample_id]
    metacell_centers = adata_train_sub.to_df().groupby(adata_train_sub.obs['metacell']).mean()

    # Sélectionner les cellules X_test du même `Id`
    adata_test_sub = adata_test[adata_test.obs['Id'] == sample_id].copy()

    if adata_test_sub.n_obs == 0:
        print(f"⚠️ Aucune cellule test pour {sample_id}, passage")
        continue

    # Trouver le plus proche voisin dans les métacellules correspondantes
    nn = NearestNeighbors(n_neighbors=1, metric="euclidean").fit(metacell_centers)
    distances, indices = nn.kneighbors(adata_test_sub.obsm['X_pca'])

    # Assigner chaque cellule X_test à une métacellule de X_train du même `Id`
    adata_test_sub.obs['metacell'] = [metacell_centers.index[i] for i in indices.flatten()]

    # 5. Calculer l'expression moyenne **par métacellule et par `Id`**
    meta_expr = adata_test_sub.to_df().groupby(['metacell']).mean()
    meta_expr['Id'] = sample_id  # Ajouter l'info `Id`
    meta_test_expression.append(meta_expr)

# 6. Concaténer et sauvegarder les résultats
meta_test_expression_df = pd.concat(meta_test_expression)
meta_test_expression_df.to_csv("meta_test_expression_by_sample.csv")

print("✅ Métacellules générées et expression agrégée sauvegardée !")